In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nsl-kdd-augmented/smote_augmented.csv
/kaggle/input/nslkdd/KDDTest+.arff
/kaggle/input/nslkdd/KDDTest-21.arff
/kaggle/input/nslkdd/KDDTest1.jpg
/kaggle/input/nslkdd/KDDTrain+.txt
/kaggle/input/nslkdd/KDDTrain+_20Percent.txt
/kaggle/input/nslkdd/KDDTest-21.txt
/kaggle/input/nslkdd/KDDTest+.txt
/kaggle/input/nslkdd/KDDTrain+.arff
/kaggle/input/nslkdd/index.html
/kaggle/input/nslkdd/KDDTrain+_20Percent.arff
/kaggle/input/nslkdd/KDDTrain1.jpg
/kaggle/input/nslkdd/nsl-kdd/KDDTest+.arff
/kaggle/input/nslkdd/nsl-kdd/KDDTest-21.arff
/kaggle/input/nslkdd/nsl-kdd/KDDTest1.jpg
/kaggle/input/nslkdd/nsl-kdd/KDDTrain+.txt
/kaggle/input/nslkdd/nsl-kdd/KDDTrain+_20Percent.txt
/kaggle/input/nslkdd/nsl-kdd/KDDTest-21.txt
/kaggle/input/nslkdd/nsl-kdd/KDDTest+.txt
/kaggle/input/nslkdd/nsl-kdd/KDDTrain+.arff
/kaggle/input/nslkdd/nsl-kdd/index.html
/kaggle/input/nslkdd/nsl-kdd/KDDTrain+_20Percent.arff
/kaggle/input/nslkdd/nsl-kdd/KDDTrain1.jpg


In [7]:
# ===========================================
# 0️⃣ Imports
# ===========================================
import pandas as pd
import numpy as np
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

# ===========================================
# 1️⃣ Load & Correct Column Assignment
# ===========================================
local_path = "/kaggle/input/nslkdd/" 

columns = [
    'duration', 'protocol_type', 'service', 'flag', 'src_bytes', 'dst_bytes', 'land',
    'wrong_fragment', 'urgent', 'hot', 'num_failed_logins', 'logged_in', 'num_compromised',
    'root_shell', 'su_attempted', 'num_root', 'num_file_creations', 'num_shells',
    'num_access_files', 'num_outbound_cmds', 'is_host_login', 'is_guest_login', 'count',
    'srv_count', 'serror_rate', 'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate',
    'same_srv_rate', 'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count',
    'dst_host_srv_count', 'dst_host_same_srv_rate', 'dst_host_diff_srv_rate',
    'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate', 'dst_host_serror_rate',
    'dst_host_srv_serror_rate', 'dst_host_rerror_rate', 'dst_host_srv_rerror_rate',
    'outcome', 'level'  # Index 41 is the attack type (outcome)
]

df_train = pd.read_csv(local_path + "KDDTrain+.txt", header=None, names=columns)
df_test  = pd.read_csv(local_path + "KDDTest+.txt", header=None, names=columns)

# Standardize to string
df_train['outcome'] = df_train['outcome'].astype(str)
df_test['outcome'] = df_test['outcome'].astype(str)

# Filter test set to only include classes present in training
train_labels = set(df_train['outcome'].unique())
df_test = df_test[df_test['outcome'].isin(train_labels)].reset_index(drop=True)

# ===========================================
# 2️⃣ Preprocessing
# ===========================================
target = 'outcome'
cat_cols = ['protocol_type', 'service', 'flag']
num_cols = [c for c in columns if c not in cat_cols + [target, 'level']]

preprocessor = ColumnTransformer([
    ('num', StandardScaler(), num_cols),
    ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), cat_cols)
])

X_train_proc = preprocessor.fit_transform(df_train.drop(columns=[target, 'level']))
X_test_proc  = preprocessor.transform(df_test.drop(columns=[target, 'level']))

le = LabelEncoder()
y_train_enc = le.fit_transform(df_train[target])
y_test_enc  = le.transform(df_test[target])
num_classes = len(le.classes_)

# ===========================================
# 3️⃣ Balanced Sampling & Weighting
# ===========================================
class_counts = np.bincount(y_train_enc)
# USE SQUARE ROOT SMOOTHING: 1/sqrt(n) is much more stable than 1/n
class_weights = 1.0 / np.sqrt(class_counts + 1)
sample_weights = class_weights[y_train_enc]

sampler = WeightedRandomSampler(weights=sample_weights, num_samples=len(y_train_enc), replacement=True)

train_dataset = torch.utils.data.TensorDataset(torch.tensor(X_train_proc, dtype=torch.float32), 
                                               torch.tensor(y_train_enc, dtype=torch.long))
test_dataset  = torch.utils.data.TensorDataset(torch.tensor(X_test_proc, dtype=torch.float32), 
                                               torch.tensor(y_test_enc, dtype=torch.long))

train_loader = DataLoader(train_dataset, batch_size=256, sampler=sampler)
test_loader  = DataLoader(test_dataset, batch_size=256, shuffle=False)

# ===========================================
# 4️⃣ Model & Loss
# ===========================================
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Normalized weights for the Loss Function
loss_weights = torch.tensor(class_weights / class_weights.sum() * num_classes, dtype=torch.float32).to(device)
criterion = nn.CrossEntropyLoss(weight=loss_weights)

class RobustMLP(nn.Module):
    def __init__(self, input_dim, num_classes):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 512),
            nn.LayerNorm(512),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, num_classes)
        )
    def forward(self, x):
        return self.net(x)

model = RobustMLP(X_train_proc.shape[1], num_classes).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

# ===========================================
# 5️⃣ Training Loop
# ===========================================
EPOCHS = 15
for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0
    for X_batch, y_batch in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    print(f"Epoch {epoch+1} Loss: {running_loss/len(train_loader):.4f}")

# ===========================================
# 6️⃣ Evaluation
# ===========================================
model.eval()
all_preds, all_labels = [], []

with torch.no_grad():
    for X_batch, y_batch in test_loader:
        outputs = model(X_batch.to(device))
        preds = torch.argmax(outputs, dim=1).cpu().numpy()
        all_preds.extend(preds)
        all_labels.extend(y_batch.numpy())

print("\n--- Evaluation Report ---")
# Use the labels parameter to ensure all classes are reported correctly
print(classification_report(all_labels, all_preds, 
                            labels=np.arange(len(le.classes_)), 
                            target_names=le.classes_, 
                            zero_division=0))

# Quick Accuracy Check
correct = (np.array(all_preds) == np.array(all_labels)).sum()
print(f"Total Accuracy: {correct / len(all_labels):.4f}")

Epoch 1: 100%|██████████| 493/493 [00:01<00:00, 295.92it/s]


Epoch 1 Loss: 0.2842


Epoch 2: 100%|██████████| 493/493 [00:01<00:00, 258.07it/s]


Epoch 2 Loss: 0.0885


Epoch 3: 100%|██████████| 493/493 [00:01<00:00, 288.61it/s]


...

Epoch 13 Loss: 0.0345


Epoch 14: 100%|██████████| 493/493 [00:01<00:00, 290.90it/s]


Epoch 14 Loss: 0.0402


Epoch 15: 100%|██████████| 493/493 [00:01<00:00, 286.21it/s]


Epoch 15 Loss: 0.0346

--- Evaluation Report ---
                 precision    recall  f1-score   support

           back       0.97      1.00      0.98       359
buffer_overflow       0.64      0.45      0.53        20
      ftp_write       0.01      0.67      0.02         3
   guess_passwd       0.61      0.02      0.03      1231
           imap       0.00      0.00      0.00         1
        ipsweep       0.80      0.99      0.88       141
           land       1.00      1.00      1.00         7
     loadmodule       0.50      1.00      0.67         2
       multihop       0.02      0.11      0.03        18
        neptune       1.00      1.00      1.00      4657
           nmap       1.00      0.99      0.99        73
         normal       0.87      0.96      0.91      9711
           perl       0.50      0.50      0.50         2
            phf       0.33      0.50      0.40         2
            pod       0.72      0.95      0.82        41
      portsweep       0.67      0.93  